In [ ]:
!pip install rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datetime import datetime as dt
from geopy.distance import great_circle
from sklearn.neighbors import BallTree
from rtree import index
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath= '/content/drive/MyDrive/CodeOp/clean_wildfires.csv'

us_wildfires = pd.read_csv(filepath)

<ipython-input-94-1a94e609a807>:3: DtypeWarning: Columns (8,9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  us_wildfires = pd.read_csv(filepath)


In [ ]:
us_wildfires["DISCOVERY_DATE"] = pd.to_datetime(us_wildfires["DISCOVERY_DATE"], format = "%m/%d/%Y")

In [ ]:
us_wildfires["CONT_DATE"] = pd.to_datetime(us_wildfires["CONT_DATE"], format = "%m/%d/%Y")

In [ ]:
us_wildfires['DURATION'] = (us_wildfires['CONT_DATE'] - us_wildfires['DISCOVERY_DATE']).dt.days

In [ ]:
us_wildfires.head()

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,DURATION
0,1,FS,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,NaN,2005-02-02,0.10,A,40.036944,-121.005833,FOREST SERVICE,CA,Plumas County,0.0
1,2,FS,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,NaN,2004-05-12,0.25,A,38.933056,-120.404444,FOREST SERVICE,CA,Placer County,0.0
2,3,FS,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,NaN,2004-05-31,0.10,A,38.984167,-120.735556,NOT SPECIFIED,CA,El Dorado County,0.0
3,4,FS,USCAENF,DEER,2004,2004-06-28,Natural,Natural,NaN,2004-07-03,0.10,A,38.559167,-119.913333,FOREST SERVICE,CA,Alpine County,5.0
4,5,FS,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,NaN,2004-07-03,0.10,A,38.559167,-119.933056,FOREST SERVICE,CA,Alpine County,5.0


In [ ]:
us_wildfires['DISCOVERY_DATE'].describe()

<ipython-input-99-0341a58cd934>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  us_wildfires['DISCOVERY_DATE'].describe()


count                 2303566
unique                  10593
top       2008-02-10 00:00:00
freq                     1220
first     1992-01-01 00:00:00
last      2020-12-31 00:00:00
Name: DISCOVERY_DATE, dtype: object

In [ ]:
us_wildfires.isna().sum()

OBJECTID                           0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
FIRE_NAME                         26
FIRE_YEAR                          0
DISCOVERY_DATE                     0
NWCG_CAUSE_CLASSIFICATION          0
NWCG_GENERAL_CAUSE                 0
NWCG_CAUSE_AGE_CATEGORY      2228039
CONT_DATE                     894813
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_DESCR                        0
STATE                              0
COUNTY                        665780
DURATION                      894813
dtype: int64

In [ ]:
us_wildfires['NWCG_CAUSE_AGE_CATEGORY'] = us_wildfires['NWCG_CAUSE_AGE_CATEGORY'].fillna("Adult or Natural")

In [ ]:
us_wildfires['FIRE_NAME'] = us_wildfires['FIRE_NAME'].fillna("UNKNOWN ")

In [ ]:
us_wildfires.isna().sum()

OBJECTID                          0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
NWCG_CAUSE_CLASSIFICATION         0
NWCG_GENERAL_CAUSE                0
NWCG_CAUSE_AGE_CATEGORY           0
CONT_DATE                    894813
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_DESCR                       0
STATE                             0
COUNTY                       665780
DURATION                     894813
dtype: int64

In [ ]:
agency_mapping = {
    'BIA': 'Bureau of Indian Affairs',
    'BLM': 'Bureau of Land Management',
    'BOR': 'Bureau of Reclamation',
    'DOD': 'Department of Defense',
    'DOE': 'Department of Energy',
    'FS': 'Forest Service',
    'FWS': 'Fish and Wildlife Service',
    'IA': 'Interagency Organization',
    'NPS': 'National Park Service',
    'ST/C&L': 'State, County, or Local Organization',
    'TRIBE': 'Tribal Organization'
}

us_wildfires['NWCG_REPORTING_AGENCY'] = us_wildfires['NWCG_REPORTING_AGENCY'].replace(agency_mapping)

us_wildfires['NWCG_REPORTING_AGENCY'].value_counts()

State, County, or Local Organization    1728168
Forest Service                           250995
Bureau of Indian Affairs                 138996
Bureau of Land Management                113672
National Park Service                     23599
Interagency Organization                  21853
Fish and Wildlife Service                 21331
Tribal Organization                        4739
Department of Defense                       163
Bureau of Reclamation                        37
Department of Energy                         13
Name: NWCG_REPORTING_AGENCY, dtype: int64

In [ ]:
(us_wildfires['STATE'] == "PR").any()

True

In [ ]:
us_wildfires_new = us_wildfires[(us_wildfires['FIRE_YEAR'] >= 2000) & (us_wildfires['STATE'] != "PR")]


In [ ]:
us_wildfires_new

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,DURATION
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,0.10,A,40.036944,-121.005833,FOREST SERVICE,CA,Plumas County,0.0
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,0.25,A,38.933056,-120.404444,FOREST SERVICE,CA,Placer County,0.0
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,0.10,A,38.984167,-120.735556,NOT SPECIFIED,CA,El Dorado County,0.0
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.913333,FOREST SERVICE,CA,Alpine County,5.0
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.933056,FOREST SERVICE,CA,Alpine County,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303561,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaT,1.00,B,39.037890,-108.959500,NOT SPECIFIED,CO,NaN,NaN
2303562,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaT,100.00,D,37.148611,-119.503056,PRIVATE,CA,Madera County,NaN
2303563,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaT,50.00,C,46.151370,-114.442800,NOT SPECIFIED,MT,NaN,NaN
2303564,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,24892.00,G,46.670340,-120.114500,FEDERAL,WA,Yakima County,3.0


In [ ]:
us_wildfires_new.isna().sum()

OBJECTID                          0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
NWCG_CAUSE_CLASSIFICATION         0
NWCG_GENERAL_CAUSE                0
NWCG_CAUSE_AGE_CATEGORY           0
CONT_DATE                    610337
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_DESCR                       0
STATE                             0
COUNTY                       339534
DURATION                     610337
dtype: int64

In [ ]:
us_counties = pd.read_csv('/content/drive/MyDrive/CodeOp/uscounties.csv')

us_counties.head()

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population
0,Los Angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3219,-118.2247,9936690
1,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5225367
2,Harris,Harris,Harris County,48201,TX,Texas,29.8578,-95.3938,4726177
3,Maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4430871
4,San Diego,San Diego,San Diego County,6073,CA,California,33.0343,-116.7351,3289701


In [ ]:
unique_lat_long_all = us_wildfires_new[['LATITUDE', 'LONGITUDE', 'COUNTY', 'STATE']].drop_duplicates()


In [ ]:
unique_lat_long_all.head()

,LATITUDE,LONGITUDE,COUNTY,STATE
0,40.036944,-121.005833,Plumas County,CA
1,38.933056,-120.404444,Placer County,CA
2,38.984167,-120.735556,El Dorado County,CA
3,38.559167,-119.913333,Alpine County,CA
4,38.559167,-119.933056,Alpine County,CA


In [ ]:
# Create R-tree index
idx = index.Index()

# Insert bounding boxes of counties into the R-tree index
for i, row in us_counties.iterrows():
    idx.insert(i, (row['lng'], row['lat'], row['lng'], row['lat']))

# Find the closest county and state for each unique latitude-longitude pair
def find_closest_county_and_state(latitudes, longitudes, default_county="Unknown", default_state="Unknown"):
    closest_counties = []
    closest_states = []
    for lat, lon in zip(latitudes, longitudes):
        nearest = list(idx.nearest((lon, lat, lon, lat), 1))
        if nearest:
            closest_counties.append(us_counties.loc[nearest[0], 'county_full'])
            closest_states.append(us_counties.loc[nearest[0], 'state_id'])
        else:
            closest_counties.append(default_county)
            closest_states.append(default_state)
    return closest_counties, closest_states

# Apply the function to get the closest counties and states for all unique lat-long pairs
unique_lat_long_all['COUNTY'], unique_lat_long_all['STATE'] = find_closest_county_and_state(unique_lat_long_all['LATITUDE'], unique_lat_long_all['LONGITUDE'])

# Merge with the original dataset to fill in the 'COUNTY' and 'STATE' columns
us_wildfires_new = pd.merge(us_wildfires_new, unique_lat_long_all[['LATITUDE', 'LONGITUDE', 'COUNTY', 'STATE']], on=['LATITUDE', 'LONGITUDE'], how='left')

# Replace the 'COUNTY' and 'STATE' columns in us_wildfires_new with the merged columns
us_wildfires_new['COUNTY'] = us_wildfires_new['COUNTY_y']
us_wildfires_new['STATE'] = us_wildfires_new['STATE_y']

# Drop the unnecessary columns
us_wildfires_new.drop(['COUNTY_y', 'STATE_y'], axis=1, inplace=True)

In [ ]:
unique_lat_long_all.head()

,LATITUDE,LONGITUDE,COUNTY,STATE
0,40.036944,-121.005833,Plumas County,CA
1,38.933056,-120.404444,El Dorado County,CA
2,38.984167,-120.735556,Placer County,CA
3,38.559167,-119.913333,Alpine County,CA
4,38.559167,-119.933056,Alpine County,CA


In [ ]:
us_wildfires_new.drop(['COUNTY_x', 'STATE_x'], axis=1, inplace=True)

In [ ]:
us_wildfires_new = us_wildfires_new.drop_duplicates()

In [ ]:
us_wildfires_new

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,0.10,A,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,0.25,A,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,0.10,A,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724950,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaT,1.00,B,39.037890,-108.959500,NOT SPECIFIED,NaN,Mesa County,CO
1724951,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaT,100.00,D,37.148611,-119.503056,PRIVATE,NaN,Madera County,CA
1724952,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaT,50.00,C,46.151370,-114.442800,NOT SPECIFIED,NaN,Ravalli County,MT
1724953,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,24892.00,G,46.670340,-120.114500,FEDERAL,3.0,Yakima County,WA


In [ ]:
us_wildfires_new['COUNTY'].value_counts()

Orange County         25284
Maricopa County       16834
Washington County     14702
Gila County           14117
Lincoln County        13806
                      ...  
St. James Parish          1
St. Charles Parish        1
Terrebonne Parish         1
Iberville Parish          1
Norfolk City              1
Name: COUNTY, Length: 1861, dtype: int64

In [ ]:
(us_wildfires_new['STATE'] == "Unknown").any()

False

In [ ]:
us_wildfires_new.isna().sum()

OBJECTID                          0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
NWCG_CAUSE_CLASSIFICATION         0
NWCG_GENERAL_CAUSE                0
NWCG_CAUSE_AGE_CATEGORY           0
CONT_DATE                    610337
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_DESCR                       0
DURATION                     610337
COUNTY                            0
STATE                             0
dtype: int64

In [ ]:
us_wildfires_new.to_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_counties.csv', index=False)

In [ ]:
us_wildfires_other = pd.read_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_counties.csv')

<ipython-input-11-b39fd86ea9c8>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  us_wildfires_other = pd.read_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_counties.csv')


In [ ]:
# Check unique values in the 'COUNTY' column for both datasets
unique_counties_wildfires = us_wildfires_other['COUNTY'].unique()
unique_counties_counties = us_counties['county_full'].unique()

# Compare unique values to identify discrepancies
discrepancies_counties = set(unique_counties_wildfires) - set(unique_counties_counties)
print("Discrepancies in counties:", discrepancies_counties)

# Repeat the same process for the 'STATE' column
unique_states_wildfires = us_wildfires_other['STATE'].unique()
unique_states_counties = us_counties['state_id'].unique()
discrepancies_states = set(unique_states_wildfires) - set(unique_states_counties)
print("Discrepancies in states:", discrepancies_states)

Discrepancies in counties: set()
Discrepancies in states: set()


In [ ]:
# Remove leading and trailing spaces from the 'COUNTY' and 'STATE' columns
us_wildfires_other['COUNTY'] = us_wildfires_other['COUNTY'].str.strip()
us_wildfires_other['STATE'] = us_wildfires_other['STATE'].str.strip()
us_counties['county_full'] = us_counties['county_full'].str.strip()
us_counties['state_id'] = us_counties['state_id'].str.strip()

# Compare cleaned values to identify discrepancies
discrepancies_counties_spaces = set(us_wildfires_other['COUNTY']) - set(us_counties['county_full'])
print("Discrepancies in counties after removing spaces:", discrepancies_counties_spaces)

discrepancies_states_spaces = set(us_wildfires_other['STATE']) - set(us_counties['state_id'])
print("Discrepancies in states after removing spaces:", discrepancies_states_spaces)

Discrepancies in counties after removing spaces: set()
Discrepancies in states after removing spaces: set()


In [ ]:
us_wildfires_other

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,0.10,A,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,0.25,A,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,0.10,A,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709361,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaN,1.00,B,39.037890,-108.959500,NOT SPECIFIED,NaN,Mesa County,CO
1709362,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaN,100.00,D,37.148611,-119.503056,PRIVATE,NaN,Madera County,CA
1709363,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaN,50.00,C,46.151370,-114.442800,NOT SPECIFIED,NaN,Ravalli County,MT
1709364,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,24892.00,G,46.670340,-120.114500,FEDERAL,3.0,Yakima County,WA


In [ ]:
us_wildfires_other = pd.merge(us_wildfires_other, us_counties[['county_full', 'state_id', 'county_fips']], how='left', left_on=['COUNTY', 'STATE'], right_on=['county_full', 'state_id'])

In [ ]:
us_wildfires_other

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,county_full,state_id,county_fips
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,...,A,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA,Plumas County,CA,6063
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,...,A,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA,El Dorado County,CA,6017
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,...,A,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA,Placer County,CA,6061
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,A,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA,Alpine County,CA,6003
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,A,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA,Alpine County,CA,6003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709361,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaN,...,B,39.037890,-108.959500,NOT SPECIFIED,NaN,Mesa County,CO,Mesa County,CO,8077
1709362,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaN,...,D,37.148611,-119.503056,PRIVATE,NaN,Madera County,CA,Madera County,CA,6039
1709363,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaN,...,C,46.151370,-114.442800,NOT SPECIFIED,NaN,Ravalli County,MT,Ravalli County,MT,30081
1709364,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,...,G,46.670340,-120.114500,FEDERAL,3.0,Yakima County,WA,Yakima County,WA,53077


In [ ]:
us_wildfires_other.isna().sum()

OBJECTID                          0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
NWCG_CAUSE_CLASSIFICATION         0
NWCG_GENERAL_CAUSE                0
NWCG_CAUSE_AGE_CATEGORY           0
CONT_DATE                    610337
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_DESCR                       0
DURATION                     610337
COUNTY                            0
STATE                             0
county_full                       0
state_id                          0
county_fips                       0
dtype: int64

In [ ]:
us_wildfires_other.drop(['state_id', 'county_full'], axis=1, inplace=True)

In [ ]:
us_wildfires_other.head()

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,county_fips
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,0.10,A,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA,6063
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,0.25,A,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA,6017
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,0.10,A,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA,6061
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA,6003
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,0.10,A,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA,6003


In [ ]:
weather_info_1 = pd.read_csv('/content/drive/MyDrive/CodeOp/test_timeseries.csv')

In [ ]:
weather_info_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2271948 entries, 0 to 2271947
Data columns (total 21 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fips         int64  
 1   date         object 
 2   PRECTOT      float64
 3   PS           float64
 4   QV2M         float64
 5   T2M          float64
 6   T2MDEW       float64
 7   T2MWET       float64
 8   T2M_MAX      float64
 9   T2M_MIN      float64
 10  T2M_RANGE    float64
 11  TS           float64
 12  WS10M        float64
 13  WS10M_MAX    float64
 14  WS10M_MIN    float64
 15  WS10M_RANGE  float64
 16  WS50M        float64
 17  WS50M_MAX    float64
 18  WS50M_MIN    float64
 19  WS50M_RANGE  float64
 20  score        float64
dtypes: float64(19), int64(1), object(1)
memory usage: 364.0+ MB


In [ ]:
columns_to_drop = ['WS10M_MIN', 'QV2M', 'T2M_RANGE', 'WS50M_MIN', 'T2M_MAX', 'WS50M', 'TS', 'WS50M_RANGE', 'WS50M_MAX', 'WS10M_MAX', 'WS10M_RANGE', 'PS', 'T2MDEW', 'T2M_MIN', 'T2MWET', 'score']

weather_info_1 = weather_info_1.drop(weather_info_1[columns_to_drop], axis=1)

In [ ]:
weather_info_1.head()

,fips,date,PRECTOT,T2M,WS10M
0,1001,2019-01-01,2.25,14.71,1.20
1,1001,2019-01-02,4.94,13.05,1.02
2,1001,2019-01-03,20.74,12.12,1.83
3,1001,2019-01-04,16.17,11.80,3.75
4,1001,2019-01-05,0.01,7.91,2.32


In [ ]:
weather_info_2 = pd.read_csv('/content/drive/MyDrive/CodeOp/train_timeseries.csv')

In [ ]:
weather_info_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19300680 entries, 0 to 19300679
Data columns (total 21 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fips         int64  
 1   date         object 
 2   PRECTOT      float64
 3   PS           float64
 4   QV2M         float64
 5   T2M          float64
 6   T2MDEW       float64
 7   T2MWET       float64
 8   T2M_MAX      float64
 9   T2M_MIN      float64
 10  T2M_RANGE    float64
 11  TS           float64
 12  WS10M        float64
 13  WS10M_MAX    float64
 14  WS10M_MIN    float64
 15  WS10M_RANGE  float64
 16  WS50M        float64
 17  WS50M_MAX    float64
 18  WS50M_MIN    float64
 19  WS50M_RANGE  float64
 20  score        float64
dtypes: float64(19), int64(1), object(1)
memory usage: 3.0+ GB


In [ ]:
weather_info_2 = weather_info_2.drop(weather_info_2[columns_to_drop], axis=1)

In [ ]:
weather_info_2.head()

,fips,date,PRECTOT,T2M,WS10M
0,1001,2000-01-01,0.22,14.74,2.20
1,1001,2000-01-02,0.20,16.69,2.52
2,1001,2000-01-03,3.65,18.49,4.03
3,1001,2000-01-04,15.95,11.40,3.84
4,1001,2000-01-05,0.00,3.86,1.60


In [ ]:
weather_info_3 = pd.read_csv('/content/drive/MyDrive/CodeOp/validation_timeseries.csv')

In [ ]:
weather_info_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268840 entries, 0 to 2268839
Data columns (total 21 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fips         int64  
 1   date         object 
 2   PRECTOT      float64
 3   PS           float64
 4   QV2M         float64
 5   T2M          float64
 6   T2MDEW       float64
 7   T2MWET       float64
 8   T2M_MAX      float64
 9   T2M_MIN      float64
 10  T2M_RANGE    float64
 11  TS           float64
 12  WS10M        float64
 13  WS10M_MAX    float64
 14  WS10M_MIN    float64
 15  WS10M_RANGE  float64
 16  WS50M        float64
 17  WS50M_MAX    float64
 18  WS50M_MIN    float64
 19  WS50M_RANGE  float64
 20  score        float64
dtypes: float64(19), int64(1), object(1)
memory usage: 363.5+ MB


In [ ]:
weather_info_3 = weather_info_3.drop(weather_info_3[columns_to_drop], axis=1)

In [ ]:
weather_info_3.head()

,fips,date,PRECTOT,T2M,WS10M
0,1001,2017-01-01,32.50,14.69,2.14
1,1001,2017-01-02,63.52,17.96,2.75
2,1001,2017-01-03,18.82,14.24,2.25
3,1001,2017-01-04,0.01,8.10,2.63
4,1001,2017-01-05,0.01,5.91,1.76


In [ ]:
combined_weather_info = pd.concat([weather_info_1, weather_info_2, weather_info_3], ignore_index=True)

In [ ]:
combined_weather_info

,fips,date,PRECTOT,T2M,WS10M
0,1001,2019-01-01,2.25,14.71,1.20
1,1001,2019-01-02,4.94,13.05,1.02
2,1001,2019-01-03,20.74,12.12,1.83
3,1001,2019-01-04,16.17,11.80,3.75
4,1001,2019-01-05,0.01,7.91,2.32
...,...,...,...,...,...
23841463,56043,2018-12-27,0.14,-9.57,2.01
23841464,56043,2018-12-28,0.02,-11.25,1.66
23841465,56043,2018-12-29,0.34,-7.17,3.64
23841466,56043,2018-12-30,3.17,-2.84,5.26


In [ ]:
us_wildfires_other.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1709366 entries, 0 to 1709365
Data columns (total 19 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   OBJECTID                   int64  
 1   NWCG_REPORTING_AGENCY      object 
 2   NWCG_REPORTING_UNIT_ID     object 
 3   FIRE_NAME                  object 
 4   FIRE_YEAR                  int64  
 5   DISCOVERY_DATE             object 
 6   NWCG_CAUSE_CLASSIFICATION  object 
 7   NWCG_GENERAL_CAUSE         object 
 8   NWCG_CAUSE_AGE_CATEGORY    object 
 9   CONT_DATE                  object 
 10  FIRE_SIZE                  float64
 11  FIRE_SIZE_CLASS            object 
 12  LATITUDE                   float64
 13  LONGITUDE                  float64
 14  OWNER_DESCR                object 
 15  DURATION                   float64
 16  COUNTY                     object 
 17  STATE                      object 
 18  county_fips                int64  
dtypes: float64(4), int64(3), object(12)
memory

In [ ]:
# Convert 'date' column in weather_data to datetime format
combined_weather_info['date'] = pd.to_datetime(combined_weather_info['date'],format="%Y-%m-%d")



In [ ]:
us_wildfires_other['DISCOVERY_DATE'] = pd.to_datetime(us_wildfires_other['DISCOVERY_DATE'],format="%Y-%m-%d")
us_wildfires_other['CONT_DATE'] = pd.to_datetime(us_wildfires_other['CONT_DATE'],format="%Y-%m-%d")


In [ ]:
# Merge the datasets based on 'DISCOVERY_DATE' and 'county_fips' in 'us_wildfires_other'
# and 'fips' and 'date' in 'weather_data'
us_wildfires_with_weather = pd.merge(us_wildfires_other, combined_weather_info, how='left', left_on=['DISCOVERY_DATE', 'county_fips'], right_on=['date', 'fips'])

# Drop redundant columns
us_wildfires_with_weather.drop(['fips', 'date'], axis=1, inplace=True)



In [ ]:
us_wildfires_with_weather

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,...,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,county_fips,PRECTOT,T2M,WS10M
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,...,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA,6063,0.01,0.49,3.76
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,...,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA,6017,0.00,13.36,2.62
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,...,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA,6061,0.41,19.16,1.80
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA,6003,0.16,19.36,1.91
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA,6003,0.16,19.36,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709361,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaT,...,39.037890,-108.959500,NOT SPECIFIED,NaN,Mesa County,CO,8077,2.12,21.78,4.76
1709362,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaT,...,37.148611,-119.503056,PRIVATE,NaN,Madera County,CA,6039,0.00,29.16,2.19
1709363,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaT,...,46.151370,-114.442800,NOT SPECIFIED,NaN,Ravalli County,MT,30081,2.74,16.00,1.29
1709364,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,...,46.670340,-120.114500,FEDERAL,3.0,Yakima County,WA,53077,0.01,26.43,3.31


In [ ]:
us_wildfires_with_weather.isna().sum()

OBJECTID                          0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
NWCG_CAUSE_CLASSIFICATION         0
NWCG_GENERAL_CAUSE                0
NWCG_CAUSE_AGE_CATEGORY           0
CONT_DATE                    610337
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_DESCR                       0
DURATION                     610337
COUNTY                            0
STATE                             0
county_fips                       0
PRECTOT                       27025
T2M                           27025
WS10M                         27025
dtype: int64

In [ ]:
nan_weather_rows = us_wildfires_with_weather[us_wildfires_with_weather['PRECTOT'].isna()]

In [ ]:
nan_weather_rows

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,...,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,county_fips,PRECTOT,T2M,WS10M
920,921,Forest Service,USSDBKF,MEADOW DRAW,2005,2005-08-07,Natural,Natural,Adult or Natural,2005-08-07,...,43.851667,-104.020833,FOREST SERVICE,0.0,Weston County,WY,56045,NaN,NaN,NaN
925,926,Forest Service,USSDBKF,REDBIRD,2005,2005-08-09,Natural,Natural,Adult or Natural,2005-08-10,...,43.772778,-104.014167,FOREST SERVICE,1.0,Weston County,WY,56045,NaN,NaN,NaN
4622,4623,Forest Service,USAKCGF,TONSINA CREEK,2005,2005-04-28,Human,Recreation and ceremony,Adult or Natural,2005-04-29,...,60.053333,-149.440000,NOT SPECIFIED,1.0,Anchorage Municipality,AK,2020,NaN,NaN,NaN
4627,4628,Forest Service,USAKCGF,SHERIDAN GLACIER FIR,2005,2005-05-01,Human,Recreation and ceremony,Adult or Natural,2005-05-02,...,60.506111,-145.343889,FEDERAL,1.0,Chugach Census Area,AK,2063,NaN,NaN,NaN
4631,4632,Forest Service,USAKCGF,PETERSON CREEK,2005,2005-05-08,Human,Fireworks,Adult or Natural,2005-05-10,...,60.885000,-149.045000,FOREST SERVICE,2.0,Anchorage Municipality,AK,2020,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703065,2297266,Bureau of Land Management,USWYNCD,TURNER,2020,2020-08-27,Natural,Natural,Adult or Natural,2020-09-10,...,43.992987,-104.479411,BUREAU OF LAND MANAGEMENT,14.0,Weston County,WY,56045,NaN,NaN,NaN
1703066,2297267,Forest Service,USWYMRF,PINEY CREEK,2020,2020-06-21,Natural,Natural,Adult or Natural,2020-06-21,...,43.645977,-104.988111,FOREST SERVICE,0.0,Weston County,WY,56045,NaN,NaN,NaN
1703077,2297278,"State, County, or Local Organization",USWYCMX,COYOTE,2020,2020-07-04,undetermined,Undetermined,Adult or Natural,NaT,...,43.676207,-105.189411,PRIVATE,NaN,Weston County,WY,56045,NaN,NaN,NaN
1709326,2303527,Bureau of Land Management,USAKUYD,EAST TIINKDHUL LAKE,2019,2019-06-22,Natural,Natural,Adult or Natural,NaT,...,66.581389,-143.277222,NOT SPECIFIED,NaN,Southeast Fairbanks Census Area,AK,2240,NaN,NaN,NaN


In [ ]:
us_wildfires_with_weather_cleaned = us_wildfires_with_weather.dropna(subset=['PRECTOT', 'T2M', 'WS10M'])

In [ ]:
us_wildfires_with_weather_cleaned

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,...,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,county_fips,PRECTOT,T2M,WS10M
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,...,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA,6063,0.01,0.49,3.76
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,...,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA,6017,0.00,13.36,2.62
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,...,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA,6061,0.41,19.16,1.80
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA,6003,0.16,19.36,1.91
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA,6003,0.16,19.36,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709361,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaT,...,39.037890,-108.959500,NOT SPECIFIED,NaN,Mesa County,CO,8077,2.12,21.78,4.76
1709362,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaT,...,37.148611,-119.503056,PRIVATE,NaN,Madera County,CA,6039,0.00,29.16,2.19
1709363,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaT,...,46.151370,-114.442800,NOT SPECIFIED,NaN,Ravalli County,MT,30081,2.74,16.00,1.29
1709364,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,...,46.670340,-120.114500,FEDERAL,3.0,Yakima County,WA,53077,0.01,26.43,3.31


In [ ]:
us_wildfires_with_weather.to_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_weather.csv', index=False)

In [ ]:
us_wildfires_with_weather_cleaned.to_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_weather_cleaned.csv', index=False)

In [ ]:
us_wildfires_complete = pd.read_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_weather_cleaned.csv')

<ipython-input-4-3d3166695002>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  us_wildfires_complete = pd.read_csv('/content/drive/MyDrive/CodeOp/us_wildfires_with_weather_cleaned.csv')


In [ ]:
elevation_data = pd.read_csv('/content/drive/MyDrive/CodeOp/soil_data.csv')

In [ ]:
elevation_data

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1005,31.870670,-85.405456,146,0.0158,0.1868,0.5441,0.2424,0.0106,0.0003,...,72.578804,1.828159,74.406960,3,2,1,1,1,1,1
2,1003,30.659218,-87.746067,52,0.0746,0.4370,0.4415,0.0469,0.0000,0.0000,...,59.843639,2.996914,62.840553,3,2,1,2,1,1,1
3,1007,33.015893,-87.127148,93,0.0144,0.1617,0.3714,0.3493,0.0898,0.0134,...,1.916593,0.008330,1.924924,3,2,1,1,1,1,1
4,1009,33.977448,-86.567246,198,0.0050,0.0872,0.2799,0.3576,0.1477,0.1037,...,1.891909,0.027488,1.919397,3,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,56037,41.660339,-108.875676,2085,0.0016,0.0281,0.1763,0.3554,0.2121,0.2097,...,0.000000,0.000000,0.000000,1,1,3,1,1,1,3
3105,56039,44.049321,-110.588102,2564,0.0003,0.0026,0.0166,0.0722,0.1489,0.5005,...,2.922309,0.000000,2.922309,1,1,1,1,1,1,1
3106,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
3107,56041,41.284726,-110.558947,2327,0.0050,0.2009,0.4063,0.1858,0.0964,0.1031,...,1.013702,10.755590,11.769293,1,1,2,1,1,1,2


In [ ]:
fips_mapping = dict(zip(elevation_data['fips'], elevation_data['elevation']))


us_wildfires_complete['ELEVATION'] = us_wildfires_complete['county_fips'].map(fips_mapping)

In [ ]:
us_wildfires_complete

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,...,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,county_fips,PRECTOT,T2M,WS10M,ELEVATION
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,...,-121.005833,FOREST SERVICE,0.0,Plumas County,CA,6063,0.01,0.49,3.76,1728
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,...,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA,6017,0.00,13.36,2.62,1442
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,...,-120.735556,NOT SPECIFIED,0.0,Placer County,CA,6061,0.41,19.16,1.80,903
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,-119.913333,FOREST SERVICE,5.0,Alpine County,CA,6003,0.16,19.36,1.91,2630
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,-119.933056,FOREST SERVICE,5.0,Alpine County,CA,6003,0.16,19.36,1.91,2630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682336,2303562,Bureau of Land Management,USCOGRD,JONES,2020,2020-06-05,Natural,Natural,Adult or Natural,NaN,...,-108.959500,NOT SPECIFIED,NaN,Mesa County,CO,8077,2.12,21.78,4.76,1479
1682337,2303563,"State, County, or Local Organization",USCAMMU,POWER,2020,2020-07-11,undetermined,Undetermined,Adult or Natural,NaN,...,-119.503056,PRIVATE,NaN,Madera County,CA,6039,0.00,29.16,2.19,633
1682338,2303564,Forest Service,USMTBRF,12 MILE,2020,2020-08-27,Natural,Natural,Adult or Natural,NaN,...,-114.442800,NOT SPECIFIED,NaN,Ravalli County,MT,30081,2.74,16.00,1.29,1817
1682339,2303565,Fish and Wildlife Service,USWAMCR,TAYLOR POND,2020,2020-08-17,Natural,Natural,Adult or Natural,2020-08-20,...,-120.114500,FEDERAL,3.0,Yakima County,WA,53077,0.01,26.43,3.31,292


In [ ]:
us_wildfires_complete.isna().sum()

OBJECTID                          0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
NWCG_CAUSE_CLASSIFICATION         0
NWCG_GENERAL_CAUSE                0
NWCG_CAUSE_AGE_CATEGORY           0
CONT_DATE                    596492
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_DESCR                       0
DURATION                     596492
COUNTY                            0
STATE                             0
county_fips                       0
PRECTOT                           0
T2M                               0
WS10M                             0
ELEVATION                         0
dtype: int64

In [ ]:
us_wildfires_complete.rename(columns={'T2M': 'TEMPERATURE',
                   'PRECTOT': 'PRECIPITATION',
                   'WS10M': 'WIND_SPEED'}, inplace=True)

In [ ]:
us_wildfires_complete.drop(['county_fips'], axis=1, inplace=True)

In [ ]:
us_wildfires_complete.head()

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,NWCG_CAUSE_AGE_CATEGORY,CONT_DATE,...,LATITUDE,LONGITUDE,OWNER_DESCR,DURATION,COUNTY,STATE,PRECIPITATION,TEMPERATURE,WIND_SPEED,ELEVATION
0,1,Forest Service,USCAPNF,FOUNTAIN,2005,2005-02-02,Human,Power supply,Adult or Natural,2005-02-02,...,40.036944,-121.005833,FOREST SERVICE,0.0,Plumas County,CA,0.01,0.49,3.76,1728
1,2,Forest Service,USCAENF,PIGEON,2004,2004-05-12,Natural,Natural,Adult or Natural,2004-05-12,...,38.933056,-120.404444,FOREST SERVICE,0.0,El Dorado County,CA,0.00,13.36,2.62,1442
2,3,Forest Service,USCAENF,SLACK,2004,2004-05-31,Human,Debris and open burning,Adult or Natural,2004-05-31,...,38.984167,-120.735556,NOT SPECIFIED,0.0,Placer County,CA,0.41,19.16,1.80,903
3,4,Forest Service,USCAENF,DEER,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,38.559167,-119.913333,FOREST SERVICE,5.0,Alpine County,CA,0.16,19.36,1.91,2630
4,5,Forest Service,USCAENF,STEVENOT,2004,2004-06-28,Natural,Natural,Adult or Natural,2004-07-03,...,38.559167,-119.933056,FOREST SERVICE,5.0,Alpine County,CA,0.16,19.36,1.91,2630


In [ ]:
us_wildfires_complete.to_csv('/content/drive/MyDrive/CodeOp/us_wildfires_complete.csv', index=False)